In [1]:
from IPython.display import clear_output

!python -m spacy download en_core_web_lg
clear_output()

In [393]:
# https://www.kaggle.com/c/whiskey-reviews-ds20/overview

import pandas as pd

# You may need to change the path
train = pd.read_csv('https://raw.githubusercontent.com/Collin-Campbell/Kaggle_whisky/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Collin-Campbell/Kaggle_whisky/main/test.csv')

In [394]:
def clean(text):

  text = text.replace('\n', '')

  text = text.replace(u'\xa0', u' ')

  text = text.lower()

  text = text.strip()


  return text

In [395]:
# Applying cleaning function to dfs

train['description'] = train['description'].apply(clean)
test['description'] = test['description'].apply(clean)

In [396]:
# A little EDA

num_of_Excellent = ((train['ratingCategory'].value_counts()[0]) / (train['ratingCategory'].value_counts().sum())) * 100
num_of_Good = ((train['ratingCategory'].value_counts()[1]) / (train['ratingCategory'].value_counts().sum())) * 100
num_of_Poor = ((train['ratingCategory'].value_counts()[2]) / (train['ratingCategory'].value_counts().sum())) * 100


print('Of the training data: \n\n\t{} percent are rated Excellent or 0 \n\t{} percent are rated Good or 1 \n\t{} percent are rated Poor or 2'.format(num_of_Excellent,num_of_Good,num_of_Poor))

Of the training data: 

	27.917788108637144 percent are rated Excellent or 0 
	70.49180327868852 percent are rated Good or 1 
	1.5904086126743333 percent are rated Poor or 2


In [397]:
# Attempting to capture context

Excellent = ["uncommon","exclusive","very flavorful","charming","very impressive","distinctive",
             "mature profile","irresistible","conspicuous","not overly so","dangerously delicious",
             "guilty pleasure","a drizzle","bliss","ripe with","is delicious","are delicious",
             "identity of its own","in balance","unpretentious","quite enjoyable",
             "lush notes","rich and sappy"," soothing","nice balance","exemplary","creamy mouthfeel",
             "carefully tempered","a luscious","editor's choice"," delight"," embraces"," silky",
             "smoky finish", " accents", "accenting","levels of complexity","coveted","undeniable",
             " classic","and rounded","a bow on"," sunset"," reveals flavors","inviting nose",
             "supple on the palate"," warming","slightly herbal","resounds","satin smooth","burnt sugars",
             "roasted lime","ripe harvest","bold and spicy","i'm enjoying","flashes of","creaminess",
             "without a doubt"," pleasant spiciness"," assertive","soothe the palate","provide balance",
             "soothes the palate","provides balance","benchmark","not aggressively"," well-made",
             "not flawed","a zip"," brightness","liquid dessert", " glorious","quite simply",
             "great mouthfeel"," deepen to"," warming"," at its best","freshness"," a pleasant mix",
             "clean and intense"," has returned","vintage","lingering"," meat","rich, dark",
             "merry mix","stronger","evenly balanced","very fragrant","with vibrant","and subtle",
             "a gentle","surprising maturity","uncommonly"]


Poor = ["dull palate","palate is dull","leaves me wondering",
        "ultimately fails","begs for","wet dog","unfortunately","could benefit",
        "shorter than expected","leaving me wondering","leaves my mouth parched",
        "leaving my mouth parched","nasty smell","ammonia","dustiness","woodiness",
        "more than one problem","more acidic and dry","hotter and drier",
        "never meant to be consumed","piercing nose","wet corn","bitter and hot",
        "inappropriate sweetness","wet oak","bad idea","no information",
        "tobacco juice","lots of pain","past its prime","spoiling","heavy palate",
        "heavy on the palate","used tea bags",
        "oversteeped tea","oversteeped mess",
        "bitter finish","finish is bitter","unpleasant finish","finish is unpleasant",
        "moonshine","fading","not a lot of substance",
        "tongue-curlingly","hot finish","finish is hot",
        "blunt alcohol","pale in color","very green","very ornery","add depth",
        "too bad","profoundly bitter","reluctantly ingesting","flinching child",
        "unwelcome spoonful","is unfortunate","not crazy about","somewhat disappointing",
        "cloying","sadly","sulphury","chewing tobacco","extremely bitter",
        "like chewing","white bread","that's about all","what is this",
        "a mess", "flabby","characterless","don't work","artificially-flavored",
        "artificially flavored","bubblegum","clashes with","too dry","too hot","too sour",
        "pleasant or enjoyable","overly sweet","too sweet","overly dry","overly hot",
        "overly sour","more distracting than","ginger ale","not a bargain","wet smell",
        "smelling a wet","watery","puckeringly","bone dry","bone-dry","is thin",
        "wet barrel","wet wood","resinous","too much oak","no balance",
        "all over the place","unrefined","solvent-like","harsh","too expensive",
        "not good enough","bubble gum","tabasco","regrettably","sticky-sweet",
        "sweet and sticky","which dominates","too dominating","rubber tire",
        "disjointed","flat finish","finish is flat","it needs more","unaged",
        "flavors flatten","flat flavor","wood dominates","flatten out","variable",
        "lifeless","very astringent","too immature","too harsh","one-dimensional",
        "long way to go","body is lacking","torpid profile","unremarkable",
        "a bit drab","flavors die","residual flavor","needs ice","veers off","potation",
        "faux-fruity","popsicles","uninspiring","fleeting finish","finish is fleeting",
        "red hots","thickly","completely masks","fun shooter","fun cocktail",
        "long and hot","a bit pulling","mashy","chill it","used bourbon cask",
        "aggressive and harsh","least favorite","soapy","medicinal","artificial",
        "yesteryear","spirity","less interesting","not particularly", "needs more",
        "needs a little more","maturing","starburst","feinty","plastic",
        "rubbery","sulphury","sweaty","young nose","dry hay","dusty grain",
        "tastes like it","unctuous","baby","children","cereal","jujubes","grape gum",
        "grain dust","wince","gripping resin","clings to","one-trick pony",
        "tailgating","barnyard","funk","fractured","there are issues","grainy aftertaste",
        "mountain dew","couch medicine","unsweetened rhubarb",
        "mouth-puckering","any redeeming qualities","no redeeming qualities","bland and saggy",
        "hard to love","out of balance","maple syrup","coke","not to savor","coca-cola",
        "pepsi","a disappointment","it is thin","it is harsh","paint thinner","maple flavoring",
        "never was meant","lacks the","mixing bourbon","on the rocks",
        "murderously","overrides","shooter","oil and water","water and oil","gamey finish",
        "finish is gamey","do not especially","badly-made","ill-conceived",
        "overtaken","far from ready","alcohol-centric","short time","..."]

       
        # Things I've noticed:
        # any comparison to childrens food is negative
        # any mention of soda or needing ice is negative
        # calling it moonshine is usually negative
        # any relation to sugary processed things is negative
        # good whisky is serious, not "fun" or for partying with
        # even punctuation matters, such as '...' vs '!'
        # exceptional reviews might not be too positive, but negative reviews are pretty negative

In [398]:
# In case of duplicates in manual recording...

Excellent = set(Excellent)
Poor = set(Poor)

In [399]:
def excellent_test(text):

  count = 0
  substrings = []

  for substring in Excellent:
    if substring in text:
      substrings.append(substring)
      count += 1
    else:
      continue

  return count#, substrings

In [400]:
def poor_test(text):

  count = 0
  substrings = []

  for substring in Poor:
    if substring in text:
      substrings.append(substring)
      count += 1
    else:
      continue

  return count#, substrings

In [401]:
# Creating new features

train['excellent_count'] = train['description'].apply(excellent_test)
train['poor_count'] = train['description'].apply(poor_test)

test['excellent_count'] = test['description'].apply(excellent_test)
test['poor_count'] = test['description'].apply(poor_test)

In [402]:
train.head(1)

,id,description,ratingCategory,excellent_count,poor_count
0,1321,"sometimes, when whisky is batched, a few lefto...",1,1,0


In [403]:
X_train = train.drop(columns=['id','ratingCategory'])
y_train = train.drop(columns=['id','description','excellent_count','poor_count'])
X_test = test.drop(columns=['id'])

In [390]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [391]:
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [392]:
submission['ratingCategory'].value_counts()

1    989
0     29
2      4
Name: ratingCategory, dtype: int64

In [342]:
subNumber = 13
submission.to_csv(f'submission{subNumber}.csv', index=False)